# Draft

In [5]:
import pandas as pd
df = pd.read_csv('globalterrorismdb_0718dist.tar.bz2', compression='bz2', low_memory=False)

In [15]:
needed_columns = [
    'iyear', 'imonth', 'iday', 'country_txt', 'region_txt', 'latitude', 'longitude',
    'attacktype1_txt', 'weaptype1_txt', 'targtype1_txt', 'targsubtype1_txt', 'target1', 'natlty1_txt', 
    'gname', 'nkill', 'nwound', 'suicide', 'success'
]
df = df[needed_columns]

Author's notes: 
- Remove Introduction headings. Replace it with just the opening.

## Overview

The **Global Terrorism Database** (GTD) is an extensive resource that tracks terrorist attacks across the world, offering data from over 200,000 incidents between 1970 and 2020. This dataset, maintained by the National Consortium for the Study of Terrorism and Responses to Terrorism (START) at the University of Maryland, serves as an invaluable tool for understanding the global patterns, causes, and consequences of terrorism. By analyzing the GTD, we gain insights into trends such as the methods, regions, and targets involved in terrorism, as well as the political, economic, and social contexts that drive these violent acts.

In this project, we leverage the GTD to explore the geographical hotspots of terrorism, uncover trends over time, and analyze the effectiveness and tactics of terrorist groups. With over four decades of data at our disposal, the goal is to identify patterns in terrorist activity, visualize global hotspots, and discuss key findings that contribute to understanding the global impact of terrorism.

The insights from this data can serve as a foundation for policymakers, security agencies, and researchers aiming to mitigate terrorism’s impact. By focusing on trends over time, geographical distribution, and methods of attack, we aim to provide a comprehensive view of the state of terrorism worldwide, offering a clear perspective on both historical and emerging trends.

In [16]:
df.head()

,iyear,imonth,iday,country_txt,region_txt,latitude,longitude,attacktype1_txt,weaptype1_txt,targtype1_txt,targsubtype1_txt,target1,natlty1_txt,gname,nkill,nwound,suicide,success
0,1970,7,2,Dominican Republic,Central America & Caribbean,18.456792,-69.951164,Assassination,Unknown,Private Citizens & Property,Named Civilian,Julio Guzman,Dominican Republic,MANO-D,1.0,0.0,0,1
1,1970,0,0,Mexico,North America,19.371887,-99.086624,Hostage Taking (Kidnapping),Unknown,Government (Diplomatic),"Diplomatic Personnel (outside of embassy, cons...","Nadine Chaval, daughter",Belgium,23rd of September Communist League,0.0,0.0,0,1
2,1970,1,0,Philippines,Southeast Asia,15.478598,120.599741,Assassination,Unknown,Journalists & Media,Radio Journalist/Staff/Facility,Employee,United States,Unknown,1.0,0.0,0,1
3,1970,1,0,Greece,Western Europe,37.997490,23.762728,Bombing/Explosion,Explosives,Government (Diplomatic),Embassy/Consulate,U.S. Embassy,United States,Unknown,NaN,NaN,0,1
4,1970,1,0,Japan,East Asia,33.580412,130.396361,Facility/Infrastructure Attack,Incendiary,Government (Diplomatic),Embassy/Consulate,U.S. Consulate,United States,Unknown,NaN,NaN,0,1
